In [10]:
import pandas as pd
import numpy as np
import datetime
import h5py

from scipy.stats import mode

In [11]:
window_width = 120  # seconds
window_stride = 60  # seconds

print("Import data")
data = pd.read_csv("capture20110810.binetflow")
# with pd.option_context('display.max_rows', None, 'display.max_columns', 15):
#    print(data.shape)
#    print(data.dtypes)
print(data.head())


Import data
                    StartTime       Dur Proto        SrcAddr Sport    Dir  \
0  2011/08/10 09:46:59.607825  1.026539   tcp  94.44.127.113  1577     ->   
1  2011/08/10 09:47:00.634364  1.009595   tcp  94.44.127.113  1577     ->   
2  2011/08/10 09:47:48.185538  3.056586   tcp   147.32.86.89  4768     ->   
3  2011/08/10 09:47:48.230897  3.111769   tcp   147.32.86.89  4788     ->   
4  2011/08/10 09:47:48.963351  3.083411   tcp   147.32.86.89  4850     ->   

        DstAddr Dport State  sTos  dTos  TotPkts  TotBytes  SrcBytes  \
0  147.32.84.59  6881  S_RA   0.0   0.0        4       276       156   
1  147.32.84.59  6881  S_RA   0.0   0.0        4       276       156   
2   77.75.73.33    80  SR_A   0.0   0.0        3       182       122   
3   77.75.73.33    80  SR_A   0.0   0.0        3       182       122   
4   77.75.73.33    80  SR_A   0.0   0.0        3       182       122   

                                    Label  
0  flow=Background-Established-cmpgw-CVUT  
1  f

In [12]:
print("Preprocessing")


def normalize_column(dt, column):
    mean = dt[column].mean()
    std = dt[column].std()
    print(mean, std)

    dt[column] = (dt[column]-mean) / std


Preprocessing


In [13]:
data['StartTime'] = pd.to_datetime(data['StartTime']).astype(np.int64)*1e-9
datetime_start = data['StartTime'].min()

data['Window_lower'] = (
    data['StartTime']-datetime_start-window_width)/window_stride+1
data['Window_lower'].clip(lower=0, inplace=True)
data['Window_upper_excl'] = (data['StartTime']-datetime_start)/window_stride+1
data = data.astype({"Window_lower": int, "Window_upper_excl": int})
data.drop('StartTime', axis=1, inplace=True)

data['Label'], labels = pd.factorize(data['Label'].str.slice(0, 15))

In [14]:
X = pd.DataFrame()  # initialize empty DataFrame
nb_windows = data['Window_upper_excl'].max()
print(nb_windows)
for i in range(nb_windows):
    gb = data.loc[(data['Window_lower'] <= i) & (
        data['Window_upper_excl'] > i)].groupby('SrcAddr')
    gb_agg = gb.agg({'Sport': 'nunique',
                     'DstAddr': 'nunique',
                     'Dport': 'nunique',
                     'Dur': ['sum', 'mean', 'std', 'max', 'median'],
                     'TotBytes': ['sum', 'mean', 'std', 'max', 'median'],
                     'SrcBytes': ['sum', 'mean', 'std', 'max', 'median'],
                     'Label': lambda x: x.mode().iat[0]})  # use mode function to get the mode
    # flatten MultiIndex column labels
    gb_agg.columns = ['_'.join(col).strip() for col in gb_agg.columns.values]
    gb_agg.reset_index(inplace=True)
    gb_agg['window_id'] = i
    X = X.append(gb_agg, ignore_index=True)

    #print(X.shape)


del (data)


368


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8992\3336780178.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(gb_agg, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8992\3336780178.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(gb_agg, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8992\3336780178.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(gb_agg, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8992\3336780178.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(gb_agg, ignore_index=True)
C:\Users\Lenovo\AppData\Loca

In [16]:
X.columns = ["_".join(x) if isinstance(x, tuple)
             else x for x in X.columns.ravel()]
# print(X.columns.values)

# std can be Nan if only one element
X.fillna(-1, inplace=True)

print(X.columns.values)
columns_to_normalize = list(X.columns.values)
columns_to_normalize.remove('SrcAddr')
columns_to_normalize.remove('Label_<lambda>')
columns_to_normalize.remove('window_id')
print(columns_to_normalize)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8992\4130396942.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  else x for x in X.columns.ravel()]


['SrcAddr' 'Sport_nunique' 'DstAddr_nunique' 'Dport_nunique' 'Dur_sum'
 'Dur_mean' 'Dur_std' 'Dur_max' 'Dur_median' 'TotBytes_sum'
 'TotBytes_mean' 'TotBytes_std' 'TotBytes_max' 'TotBytes_median'
 'SrcBytes_sum' 'SrcBytes_mean' 'SrcBytes_std' 'SrcBytes_max'
 'SrcBytes_median' 'Label_<lambda>' 'window_id']
['Sport_nunique', 'DstAddr_nunique', 'Dport_nunique', 'Dur_sum', 'Dur_mean', 'Dur_std', 'Dur_max', 'Dur_median', 'TotBytes_sum', 'TotBytes_mean', 'TotBytes_std', 'TotBytes_max', 'TotBytes_median', 'SrcBytes_sum', 'SrcBytes_mean', 'SrcBytes_std', 'SrcBytes_max', 'SrcBytes_median']


In [17]:
pip install tables


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:


normalize_column(X, columns_to_normalize)

with pd.option_context('display.max_rows', 10, 'display.max_columns', 22):
    print(X.shape)
    print(X)
    print(X.dtypes)

# with pd.option_context('display.max_rows', 10, 'display.max_columns', 20):
#    print(X.loc[X['Label'] != 0])

X.drop('SrcAddr', axis=1).to_hdf(
    'model_pp1.h5', key="data", mode="w")
np.save("ids.npy", X['SrcAddr'])
np.save("labels.npy", labels)


Sport_nunique          2.172287
DstAddr_nunique        1.312422
Dport_nunique          1.287559
Dur_sum             1090.604709
Dur_mean             791.165306
Dur_std               11.024080
Dur_max              806.506942
Dur_median           789.858249
TotBytes_sum       48577.964323
TotBytes_mean       7749.940752
TotBytes_std        7523.617775
TotBytes_max       37139.062650
TotBytes_median     6488.193718
SrcBytes_sum       10215.123229
SrcBytes_mean       3353.451307
SrcBytes_std        3037.116941
SrcBytes_max        8784.021176
SrcBytes_median     2910.874682
dtype: float64 Sport_nunique      4.260579e+01
DstAddr_nunique    1.241158e+01
Dport_nunique      6.163165e+01
Dur_sum            1.923882e+04
Dur_mean           1.234872e+03
Dur_std            1.329874e+02
Dur_max            1.247477e+03
Dur_median         1.236083e+03
TotBytes_sum       5.815655e+06
TotBytes_mean      1.585917e+06
TotBytes_std       2.245570e+06
TotBytes_max       4.958437e+06
TotBytes_median    1.5528